# 🚀 FinSentiment 3.0: Sector Screener (Batch Edition)

Versión escalable para analizar múltiples empresas del sector tecnológico de forma secuencial y comparativa.

### Nuevas Características (Fase 7):
1. **Batch Processing**: Análisis automático de ['META', 'AAPL', 'MSFT', 'GOOGL', 'AMZN'].
2. **Robustez**: Manejo de errores encapsulado (si falla uno, sigue el siguiente).
3. **Screener Visual**: Ranking de sentimiento (Z-Score) para comparar empresas.
4. **Core v2.1**: Filtros legales y normalización estadística ya integrados.

In [ ]:
# @title 1. Instalación de Dependencias
!pip install sec-edgar-downloader transformers torch pandas numpy matplotlib seaborn beautifulsoup4 yfinance

In [ ]:
# @title 2. Definición del Motor (Core Engine v2.1)

import os
import re
import glob
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
from collections import Counter
from datetime import datetime, timedelta
from bs4 import BeautifulSoup
from sec_edgar_downloader import Downloader
from transformers import BertTokenizer, BertForSequenceClassification

# Configurar estilos
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = [14, 7]

# --- MÓDULO 1: INGESTA ---
class SECLoader:
    def __init__(self, data_dir="data", email="research@example.com", company="Personal Research"):
        self.data_dir = data_dir
        os.makedirs(os.path.join(data_dir, "raw"), exist_ok=True)
        os.makedirs(os.path.join(data_dir, "processed"), exist_ok=True)
        self.downloader = Downloader(company, email, os.path.join(data_dir, "raw"))

    def download_filings(self, ticker: str, amount: int = 1):
        path = os.path.join(self.data_dir, "raw", "sec-edgar-filings", ticker)
        # En batch mode, a veces conviene forzar descarga si amount es bajo
        print(f"[>] ({ticker}) Iniciando descarga de último reporte...")
        try:
            self.downloader.get("10-K", ticker, limit=amount)
        except Exception as e:
            print(f"[!] Error en descarga {ticker}: {e}")

    def extract_date(self, content: str) -> str:
        patterns = [
            r'FILED AS OF DATE:\s+(\d{8})',
            r'CONFORMED PERIOD OF REPORT:\s+(\d{8})'
        ]
        for p in patterns:
            match = re.search(p, content)
            if match:
                date_str = match.group(1)
                return f"{date_str[:4]}-{date_str[4:6]}-{date_str[6:]}"
        return None

    def extract_mda(self, html_content: str) -> str:
        soup = BeautifulSoup(html_content, 'html.parser')
        text = soup.get_text(separator='\n')
        patterns = [r'Item\s+7\.\s+Management', r"Management's\s+Discussion", r'Item\s+7\.']
        start_idx = -1
        for p in patterns:
            match = re.search(p, text, re.IGNORECASE)
            if match: start_idx = match.start(); break
        if start_idx == -1: return text[:50000]
        return text[start_idx:start_idx+30000]

    def process_filings(self, ticker: str):
        raw_path = os.path.join(self.data_dir, "raw", "sec-edgar-filings", ticker)
        processed_data = []
        for root, _, files in os.walk(raw_path):
            for file in files:
                if file.lower().endswith(".txt") and "primary" not in file:
                    try:
                        with open(os.path.join(root, file), 'r', encoding='utf-8', errors='ignore') as f:
                            content = f.read()
                        mda = self.extract_mda(content)
                        date = self.extract_date(content)
                        if len(mda) > 500:
                            processed_data.append({'text': mda, 'date': date, 'accession': file})
                    except: pass
        # Ordenar por fecha reciente
        return sorted(processed_data, key=lambda x: x.get('date', '1900'), reverse=True)

# --- MÓDULO 2 NOISE FILTER ---
class TextPreprocessor:
    def clean_text(self, text):
        text = re.sub(r'\s+', ' ', text)
        return "".join(ch for ch in text if ch.isprintable()).strip()
    
    def is_legal_noise(self, sentence: str) -> bool:
        legal_keywords = [
            'forward-looking', 'safe harbor', 'uncertainty', 'may differ', 
            'subject to error', 'actual results', 'factors that could cause',
            'statements regarding', 'cautionary note', 'risk factors', 
            'include but are not limited to', 'assumptions'
        ]
        return any(kw in sentence.lower() for kw in legal_keywords)

    def split_sentences(self, text):
        sentences = re.split(r'(?<=[.!?])\s+(?=[A-Z])', text)
        valid_sentences = []
        for s in sentences:
            s = s.strip()
            if len(s) > 20 and len(s.split()) >= 4:
                if not self.is_legal_noise(s):
                    valid_sentences.append(s)
        return valid_sentences

# --- MÓDULO 3 FINBERT ---
class FinBertModel:
    def __init__(self):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.tokenizer = BertTokenizer.from_pretrained("ProsusAI/finbert")
        self.model = BertForSequenceClassification.from_pretrained("ProsusAI/finbert").to(self.device)
        self.labels = {0: 'positive', 1: 'negative', 2: 'neutral'}

    def predict(self, sentences):
        if not sentences: return pd.DataFrame()
        batch_size = 32
        results = []
        for i in range(0, len(sentences), batch_size):
            batch = sentences[i:i+batch_size]
            inputs = self.tokenizer(batch, return_tensors="pt", padding=True, truncation=True, max_length=512).to(self.device)
            with torch.no_grad():
                outputs = self.model(**inputs)
                probs = torch.nn.functional.softmax(outputs.logits, dim=-1).cpu().numpy()
            for j, s in enumerate(batch):
                results.append({
                    "sentence": s,
                    "pos_val": probs[j][0],
                    "neg_val": probs[j][1]
                })
        return pd.DataFrame(results)


In [ ]:
# @title 3. Ejecución Batch (Multi-Ticker)

# Inicializar Modelos fuera del loop para eficiencia
print("[*] Inicializando Motor NLP...")
loader = SECLoader()
model = FinBertModel()
prep = TextPreprocessor()

def analyze_ticker(ticker_symbol):
    """Función Maestra: Procesa una empresa de principio a fin."""
    print(f"\n--- Procesando {ticker_symbol} ---")
    try:
        # 1. Download
        loader.download_filings(ticker_symbol, amount=1)
        
        # 2. Process
        docs = loader.process_filings(ticker_symbol)
        if not docs:
            print(f"[!] No se encontraron reportes legibles para {ticker_symbol}")
            return None
            
        latest_doc = docs[0] # Tomamos el más reciente
        report_date = latest_doc.get('date', 'Unknown')
        print(f"[>] Analizando reporte del {report_date}...")
        
        # 3. Clean & Predict
        sentences = prep.split_sentences(prep.clean_text(latest_doc['text']))
        df = model.predict(sentences)
        
        if df.empty:
            return None
            
        # 4. Calculate Metrics
        net_score = df['pos_val'].mean() - df['neg_val'].mean()
        
        # Z-Score muy aproximado (usando una desviación estándar teórica del sector si no hay historial)
        # Para ser justos en una comparativa transversal, usamos el Score Neto directo o un Z-Score vs sus oraciones.
        # Aquí calcularemos Z-Score interno de las oraciones para ver consistencia.
        # PERO para el Screener Visual, el usuario pidió Z-Score.
        # Como es un solo punto de dato por empresa, es difícil calcular Z-Score histórico en una sola corrida rápida.
        # Simularemos Z-Score normalizando contra una media 'neutral' (0).
        
        return {
            'ticker': ticker_symbol,
            'date': report_date,
            'net_score': net_score
        }
        
    except Exception as e:
        print(f"[ERROR] Fallo crítico en {ticker_symbol}: {e}")
        return None

# --- BUCLE PRINCIPAL ---
TICKERS = ['META', 'AAPL', 'MSFT', 'GOOGL', 'AMZN']
sector_results = []

for t in TICKERS:
    res = analyze_ticker(t)
    if res:
        sector_results.append(res)

# Construir DataFrame
df_sector = pd.DataFrame(sector_results)

if not df_sector.empty:
    # Calcular Z-Score relativo al GRUPO (Sectorial)
    # Esto nos dice quién está mejor/peor que sus pares HOY.
    sector_mean = df_sector['net_score'].mean()
    sector_std = df_sector['net_score'].std()
    if sector_std == 0: sector_std = 1
    
    df_sector['z_score'] = (df_sector['net_score'] - sector_mean) / sector_std
    
    print("\n" + "="*40)
    print(" SECTOR SCREENER RESULTS")
    print("="*40)
    display(df_sector.sort_values('z_score', ascending=False))
    
    # --- VISUALIZACIÓN COMPARATIVA ---
    plt.figure(figsize=(10, 6))
    
    # Colores divergentes: Verde para positivo, Rojo para negativo
    colors = ['#2ecc71' if x >= 0 else '#e74c3c' for x in df_sector['z_score']]
    
    sns.barplot(x='z_score', y='ticker', data=df_sector.sort_values('z_score', ascending=False), palette=colors)
    
    plt.title('Market Sentiment Screener: Tech Sector (Z-Score Relativo)', fontsize=15, fontweight='bold')
    plt.xlabel('Z-Score (Desviación respecto al promedio del grupo)', fontsize=12)
    plt.ylabel('Ticker', fontsize=12)
    plt.axvline(0, color='black', linestyle='--', alpha=0.3)
    plt.grid(axis='x', alpha=0.3)
    plt.show()
else:
    print("[!] No se obtuvieron resultados válidos para generar el gráfico.")